In [ ]:
import pandas as pd


In [ ]:
world_data = pd.read_csv("data/world_bank/combined_world_data.csv")
eurovision_data = pd.read_csv("data/eurovision/cleansed_eurovision_data.csv")

In [ ]:
world_data.sample(10)

In [ ]:
eurovision_data.sample(10)

In [ ]:
eurovision_countries = eurovision_data["Country"].unique()
world_countries = world_data["Country Name"].unique()

In [ ]:
for country in eurovision_countries:
    if country not in world_data["Country Name"].unique():
        print(country)

In [ ]:
for country in world_countries:
    if country not in eurovision_countries:
        print(country)

In [ ]:
eurovision_data["Country"].replace({"Czech Republic": "Czechia", "Russia": "Russian Federation", "Slovakia": "Slovak Republic", "Turkey": "Turkiye"}, inplace=True)

In [ ]:
eurovision_data.rename(columns={"Country": "Country Name"}, inplace=True)

In [ ]:
merged_df = pd.merge(eurovision_data, world_data, on=['Country Name', 'Year'], how='inner').reset_index()


In [ ]:
first_cols = ['Country Name', 'Country Code', 'Year', "Song", "Artist", "Language", "Grand Final Place", "Grand Final Points", "Semifinal", "Semifinal Place", "Semifinal Points"]
remaining_cols = [col for col in merged_df.columns if col not in first_cols]
new_col_order = first_cols + remaining_cols

In [ ]:
merged_df = merged_df[new_col_order]

In [ ]:
merged_df.tail(10)

In [ ]:
merged_df.info()

In [ ]:
merged_df.to_csv("data/combined_data.csv")